# 🤖 Formation RAG – Partie 1

---
## 🎯 Objectifs pédagogiques

Cette formation a pour but de vous initier au concept de **RAG (Retrieval-Augmented Generation)**. À la fin de cette formation, vous serez capable de :

- Comprendre les composants essentiels d’un système RAG
- Manipuler du code Python sur Colab
- Installer et utiliser **Ollama** pour faire tourner un modèle LLM localement
- Tester un prototype sur un document PDF
- Explorer les étapes vers une mise en production (abordées en deuxième partie)


# 🚧 Séquence 1.0 – Setup du Projet (Expliqué en partie 2)

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
import subprocess, time
ollama_proc = subprocess.Popen("ollama serve", shell=True)
time.sleep(2)
print('✅ Ollama est prêt')

In [ ]:
!ollama pull llama3.2:latest
!ollama pull nomic-embed-text:latest
!ollama pull bge-m3

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/MyDrive
!git clone https://github.com/antoinecstl/FormationEYAI.git
%cd FormationEYAI

In [ ]:
!pip install -r requirements.txt

# 🔍 Séquence 1.1 – Bases du RAG : embeddings & similarité (SLIDE)

## 🧠 Création d'embeddings avec un modèle local

Pour pouvoir comparer des phrases ou retrouver des documents pertinents, on doit **transformer du texte en vecteurs numériques** (embeddings). Ces vecteurs capturent le sens des mots ou des phrases dans un espace mathématique.

### 🔧 Que fait cette cellule ?
- Elle définit une fonction `embed_texts` qui prend une **liste de phrases** en entrée.
- Chaque phrase est transformée en vecteur via le modèle `nomic-embed-text` installé localement avec Ollama. (Nous préparons ici deux modèles d'embeeding `nomic-embed-text` et `bge-m3`que nous utiliserons par la suite)
- Elle retourne un tableau `numpy` contenant les vecteurs (`shape = (n, d)`), où :
  - `n` est le nombre de phrases
  - `d` est la dimension de l’espace d’embedding

Ces vecteurs seront utiles pour calculer des similarités ou faire de la recherche sémantique.


In [ ]:
import numpy as np, ollama

EMBED_MODEL1 = "nomic-embed-text:latest"
EMBED_MODEL2 = "bge-m3"

def embed_texts(texts, embed_model):
    """Retourne un np.ndarray shape (n, d)"""
    return np.array([ollama.embeddings(model=embed_model, prompt=t)['embedding'] for t in texts], dtype='float32')

## 🔬 Test d'embedding : comparaison de phrases

Ici, on mesure la **similarité** entre deux phrases à l’aide de leurs embeddings.

#### 🔍 Que fait cette cellule ?
- Elle convertit chaque phrase en vecteur (embedding)
- Elle mesure leur proximité à l’aide d’un **produit scalaire**
- Le score obtenu indique le **niveau de similarité sémantique**

#### 📊 Comment lire le score ?
- `1` : phrases très proches (sens similaire)
- `0` : phrases sans lien
- `-1` : phrases opposées (Les modèles que nous utilisons sont très générique et n'arrivent que très rarement à aller en dessous de 0. En effet, les embeddings de modèles récents sont faits pour maximiser la similarité entre phrases proches, pas pour maximiser la dissimilarité.)

✏️ Vous pouvez modifier les phrases pour tester différents cas.

In [ ]:
phrase1 = "Cite moi les meilleures écurie de Formule 1"
phrase2 = "Pourquoi le ciel est bleu ?"

vecs = embed_texts([phrase1, phrase2], EMBED_MODEL1)
sim = float(vecs[0] @ vecs[1] / (np.linalg.norm(vecs[0])*np.linalg.norm(vecs[1])))
print(f"Similarité : {sim:.3f}")

# 📐 Séquence 1.2 – Chunking & nettoyage d’un PDF d’exemple (SLIDE)

#### 🔍 Que fait cette cellule ?

Ici, ce bloc de code sert à découper un texte en morceaux adaptés au LLM. 

La fonction `auto_chunk_size` détermine la taille des morceaux (chunks) en fonction de la longueur du texte, afin de rester efficace. Plus le texte est long et plus les chunks sont petit.

La fonction `chunk_document` découpe un texte en morceaux exploitables par un LLM.
    1. Calcule la taille des chunks avec `auto_chunk_size`.
    2. Créer un découpeur intelligent `splitter` qui coupe aux paragraphes, puis lignes, puis phrases.
    3. Filtre les morceaux trop courts (< 100 caractères>)

Cela permet de faciliter l'analyse de texte long par le LLM en préservant le sens.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def auto_chunk_size(tok:int)->int:
    return 1024 if tok<8000 else 768 if tok<20000 else 512

def chunk_document(text:str):
    size=auto_chunk_size(len(text.split()))
    splitter=RecursiveCharacterTextSplitter(
        separators=["\n\n","\n",". "],
        chunk_size=size,
        chunk_overlap=size//4,
        length_function=len,
        )
    return [c for c in splitter.split_text(text) if len(c)>100]

## 📖 Chargement du PDF d'exemple

On commence par **extraire le texte brut du PDF** page par page grâce à la librairie PyPDF2.

### 💡 Pourquoi faire ça ?  
- Cela permet de récupérer tout le contenu textuel du document.  
- On peut ensuite estimer sa taille en nombre de mots (tokens), ce qui aide à adapter les traitements (chunking, embeddings, etc.).

In [ ]:
from PyPDF2 import PdfReader

sample_path = "/content/gdrive/MyDrive/FormationEYAI/Anonymized_Rapport.pdf"
pages = PdfReader(sample_path).pages
full_text = "\n".join(p.extract_text() or "" for p in pages)

print(f"📄 Le document contient {len(full_text.split())} tokens environ.")

### 🌳 Chunking du PDF
Ici, le text extrait du pdf est découpé en plus petit segments (chunks), afin de préparer le texte pour de l'indexation.


### Pourquoi créer des chunks ?  
- Les modèles ne peuvent pas traiter de très longs textes d’un coup.  
- Le chunking permet de diviser le contenu en morceaux cohérents et exploitables.  
- On peut ensuite traiter chaque chunk indépendamment (calcul d’embeddings, recherche, etc.).


✏️ Parcourer la lise de chunks générés afin de valider le bon découpage du contenu.

In [ ]:
chunks = chunk_document(full_text)
print(f"{len(chunks)} chunks créés.\nAperçu :\n{chunks[0][:300]}…")

# 📊 Séquence 1.3 – Index vectoriel FAISS (SLIDE)


Les vecteurs obtenus à partir des chunks sont rangés dans une structure appelée **index FAISS**.

### Qu’est-ce que FAISS ?  
- Un outil très rapide pour rechercher les vecteurs proches dans un grand ensemble.  
- Permet de retrouver rapidement les documents les plus similaires à une requête.

### 💡 Pourquoi créer cet index ?  
- Pour accélérer les recherches dans la base de documents vectorisés.  
- C’est indispensable dès qu’on a beaucoup de données à parcourir.

In [ ]:
import faiss, numpy as np

def build_faiss_index(vectors:np.ndarray)->faiss.IndexFlatIP:
    d=vectors.shape[1]
    idx=faiss.IndexFlatIP(d)
    idx.add(vectors.astype('float32'))
    return idx

## 🧪 Construction index chunks


Chaque chunk est transformé en vecteur numérique (embedding), puis ajouté à l’index FAISS.

### Ce que ça signifie :  
- On passe de textes à vecteurs.  
- On construit une base efficace pour retrouver les chunks les plus pertinents rapidement.

In [ ]:
import numpy as np

chunk_vecs = embed_texts(chunks, EMBED_MODEL2)
index = build_faiss_index(chunk_vecs)

print(index.ntotal, "Nombre de vecteurs total dans l'index")

# 🧮 Séquence 1.4 – Algorithme MMR

MMR permet de sélectionner des passages à la fois **pertinents** et **diversifiés** pour une requête donnée.

### Pourquoi c’est important ?  
- Sélectionner uniquement les passages les plus similaires peut donner des résultats redondants.  
- MMR équilibre la similarité à la question et la diversité entre passages sélectionnés.

Cet algorithme améliore la qualité des résultats en évitant les répétitions.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def mmr(query_vec:np.ndarray, cand:np.ndarray, k:int=5, λ:float=0.3):
    selected, rest = [], list(range(len(cand)))
    while len(selected)<min(k,len(rest)):
        best, best_score = None, -1e9
        for idx in rest:
            sim_q = float(query_vec @ cand[idx]/(np.linalg.norm(query_vec)*np.linalg.norm(cand[idx])+1e-6))
            sim_s = max(cosine_similarity(cand[idx][None,:], cand[selected])[0]) if selected else 0.
            score = λ*sim_q - (1-λ)*sim_s
            if score>best_score:
                best, best_score = idx, score
        selected.append(best); rest.remove(best)
    return selected

## 🔬 Test MMR
Ici, l'algorithme MMR est appelé pour trouver les 3 passages les plus pertinents et variés en réponse à la question.

✏️Modifier la question pour s'assurer que MMR sélectionne bien des passages différents mais liés à la question.

In [ ]:
import numpy as np

q_vec = embed_texts(["Sujet principal du rapport ?"], EMBED_MODEL2)[0]
sel = mmr(q_vec, chunk_vecs, 3)
print(sel)

# 🧑‍🎤 Séquence 1.5 – Prompt engineering (SLIDE)

## 🧠 Construction du prompt

Après avoir sélectionné les passages du document les plus pertinents et variés par rapport à la question,  
on construit un prompt clair et structuré avec `build_prompt`.

`ctxs` représente les informations du document jugées pertinentes à la `question` posée.

### 💡 Pourquoi on fait ça ?  
Cette étape est essentielle pour que le modèle fournisse une réponse ciblée et fiable,  
en s’appuyant uniquement sur les données extraites du document.

In [ ]:
def build_prompt(question:str, ctxs:list[str]):
    ctx_block="\n\n".join(f"[{i+1}] {c}" for i,c in enumerate(ctxs))
    system="Vous êtes un assistant expert. Utilisez uniquement les informations suivantes pour répondre en français. Citez les sources [n]."
    return [
        {"role":"system","content":system},
        {"role":"user","content":f"CONTEXTE(S):\n{ctx_block}\n\nQUESTION: {question}\n\nRéponse:"}
    ]


In [ ]:
print(build_prompt("Pourquoi le ciel est bleu ?", ["La diffusion Rayleigh explique la couleur du ciel."]))

## 🤖 Séquence 1.6 Premier Appel au LLM (SLIDE)

Ce code envoie une liste de messages au modèle `llama3.2:3B` via la fonction `_call_llm`.

- Le premier message définit le rôle ou le comportement attendu du modèle.  
- Le second contient la question posée.

On peut ajuster la créativité (`temperature`) et la longueur de la réponse (`max_tokens`).

✏️ Modifie ces paramètres et la question pour tester et comprendre l’impact sur les réponses.


In [ ]:
from typing import List, Dict

MODEL_NAME = "llama3.2:3B"

def _call_llm(messages: List[Dict[str, str]], *, temperature: float = 0.1, max_tokens: int = 2048, stream: bool = False):
    """Enveloppe simple autour de ollama.chat pour usage direct."""
    return ollama.chat(
        model=MODEL_NAME,
        messages=messages,
        stream=stream,
        options={"temperature": temperature, "num_predict": max_tokens},
    )

# 🧪 Exemple d'appel
messages = [
    {"role": "system", "content": "Tu es un assistant concis"},
    {"role": "user", "content": "Donne-moi la capitale de l’Italie"}
]
print(_call_llm(messages)["message"]["content"].strip())

# 🏗️ Séquence 1.7 – Assemblage mini‑RAG (prototype)

Ici, on combine toutes les étapes vues précédemment pour créer un système simple de RAG qui répond à une question à partir d’un document.

1. Transformer la question en vecteur (embedding) avec `embed_texts`.  
2. Chercher les passages les plus proches dans l’index FAISS (`index.search`).  
3. Récupérer les textes correspondants à ces passages.  
4. Construire un prompt structuré avec `build_prompt`.  
5. Appeler le modèle de langage avec `_call_llm` pour générer la réponse.  
6. Retourner la réponse et les passages utilisés.

Cete fonction montre comment utiliser les embeddings et l’indexation pour alimenter un LLM en contexte précis.

In [ ]:
def ask(question: str, chunks: List[str], vecs: np.ndarray, top_k: int = 3):
    # Recherche des chunks pertinents
    q_vec = embed_texts([question], EMBED_MODEL2)[0]
    _, I = index.search(q_vec[None, :], top_k)
    ctx = [chunks[i] for i in I[0]]
    # Préparation du prompt
    prompt = build_prompt(question, ctx)
    # Appel LLM et retour de la réponse
    answer = _call_llm(prompt)["message"]["content"].strip()
    return answer, I[0]

## 🧪 Prototype RAG sur le PDF


Maintenant, on peut poser une question sur le PDF `rapport.pdf` et obtenir une réponse sourcée basée sur le contenu réel du document.

✏️ Change la question pour explorer différentes réponses !



In [ ]:
question = "Qui est le prestataire de la mission ?"
print(ask(question, chunks, chunk_vecs))